# imports

## pip

In [3]:
# !pip install anchor-exp
# !pip install pmlb
# !pip install z3-solver

## import

In [4]:
import numpy as np
import pandas as pd

from anchor import utils
from anchor import anchor_tabular
import sklearn
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from pmlb import fetch_data
from z3 import *

In [59]:
set_option(rational_to_decimal=True)

# model

In [5]:
def feature_constraints_expression(X):
    constraints = []

    for i in range(X.shape[1]):
        feature_values = X[:, i]
        min_val, max_val = feature_values.min(), feature_values.max()

        x = Real(f'x{i}')
        min = RealVal(min_val)
        max = RealVal(max_val)

        constraint = And(min <= x, x <= max)
        constraints.append(constraint)

    return And(*constraints)

In [6]:
def tree_paths_expression(tree, tree_index, class_index):
    tree_ = tree.tree_
    feature = tree_.feature
    threshold = tree_.threshold
    value = tree_.value

    paths = []
    o = Real(f'o_{tree_index}_{class_index}')

    def traverse(node, path_conditions):

        if feature[node] == -2:
            leaf_value = value[node][0][0]
            path_formula = And(path_conditions)
            implication = Implies(path_formula, o == leaf_value)
            paths.append(implication)
        else:

            x = Real(f'x{feature[node]}')
            left_condition = x <= threshold[node]
            right_condition = x > threshold[node]
            traverse(tree_.children_left[node],
                     path_conditions + [left_condition])
            traverse(tree_.children_right[node],
                     path_conditions + [right_condition])

    traverse(0, [])
    return And(*paths)

In [7]:
def model_trees_expression(model):
    formulas = []
    for i, estimators in enumerate(model.estimators_):
        for class_index, estimator in enumerate(estimators):
            formula = tree_paths_expression(estimator, i, class_index)
            formulas.append(formula)
    return And(*formulas)

In [8]:
def decision_function_expression(model, x):
    learning_rate = model.learning_rate
    estimators = model.estimators_
    n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_

    decision = model.decision_function(x)
    predicted_class = model.predict(x)[0]

    estimator_results = []
    for estimator in estimators:
        class_predictions = [tree.predict(x) for tree in estimator]
        estimator_results.append(class_predictions)

    estimator_sum = np.sum(estimator_results, axis=0) * learning_rate
    init_value = decision - estimator_sum.T

    equation_list = []
    for class_number in range(n_classes):
        estimator_list = []
        for estimator_number in range(len(estimators)):
            o = Real(f"o_{estimator_number}_{class_number}")
            estimator_list.append(o)
        equation_o = Sum(estimator_list) * learning_rate + init_value[0][class_number]
        equation_list.append(equation_o)

    if n_classes <= 2:
        if predicted_class == 0:
            final_equation = equation_list[0] < 0
        else:
            final_equation = equation_list[0] > 0
    else:
        compare_equation = []
        for class_number in range(n_classes):
            if predicted_class != class_number:
                compare_equation.append(
                    equation_list[predicted_class] > equation_list[class_number]
                )
        final_equation = compare_equation

    return And(final_equation)

In [9]:
def instance_expression(instance):
    formula = [Real(f'x{i}') == value for i, value in enumerate(instance)]
    return formula

In [10]:
def is_proved(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        return True
    else:
        # print(s.model())
        return False

In [11]:
def explain(I, T, D, model, reorder):
    X = I.copy()
    T_s = simplify(T)
    D_s = simplify(D)

    importances = model.feature_importances_
    if reorder == 'asc':
        sorted_feature_indices = np.argsort(importances)
        X = [X[i] for i in sorted_feature_indices]
    elif reorder == 'desc':
        sorted_feature_indices = np.argsort(np.flip(importances))
        X = [X[i] for i in sorted_feature_indices]

    for feature in X.copy():
        X.remove(feature)

        # prove(Implies(And(And(X), T), D))
        if is_proved(Implies(And(And(X), T_s), D_s)):
            continue
            # print('proved')
        else:
            # print('not proved')
            X.append(feature)

    return X

In [12]:
class Explainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data
        self.T_constraints = feature_constraints_expression(self.data)
        self.T_model = model_trees_expression(self.model)
        self.T = And(self.T_model, self.T_constraints)

    def explain(self, instance, reorder='asc'):
        self.D = decision_function_expression(self.model, [instance])
        self.I = instance_expression(instance)

        return explain(self.I, self.T, self.D, self.model, reorder)

In [74]:
class ExplainerCompleter():
  def __init__(self, model, anchor_explainer, data, round):
    self.model = model

    # anchor
    # explain instance > matriz > expressions
    self.anchor_explainer = anchor_explainer

    # model
    # T
    self.T_constraints = feature_constraints_expression(data)
    self.T_model = model_trees_expression(self.model)
    self.T = And(self.T_model, self.T_constraints)

  def explain_instance(self, instance, verbose=False, test=0):
    opt = Optimize()

    # anchor matrix > expressions
    self.exp = self.anchor_explainer.explain_instance(instance, self.model.predict, threshold=0.95)
    anchor_matrix = []
    for name in self.exp.names():
      tokens = name.split(' ')
      for operator in ['<=', '>=', '==', '<', '>']:
        if operator in name:
          parts = name.split(operator)
          if len(parts) == 2:
            anchor_matrix.append([parts[0].strip(), operator, parts[1].strip()])
            break
    # unir com o código de cima para simplificar
    anchor_expressions = []
    for row in anchor_matrix:
      feature = Real(row[0])
      if row[1] == '<=':
        expression = feature <= float(row[2])
      elif row[1] == '>=':
        expression = feature >= float(row[2])
      elif row[1] == '<':
        expression = feature < float(row[2])
      elif row[1] == '>':
        expression = feature > float(row[2])
      anchor_expressions.append(expression)
    # print(anchor_expressions, len(anchor_expressions) == len(anchor_matrix))
    self.anchor_expressions = anchor_expressions
    opt.add(anchor_expressions)

    # delta
    # delta >= 0
    # todas as features que não estao no anchor > fazer as igualdades delta
    anchor_variables = []
    for formula in anchor_expressions:
      anchor_variables.append(str(formula.arg(0)))

    delta = Real('delta')
    if test > 0:
      opt.add(delta <= test)
    feature_names = [f'x{i}' for i in range(instance.shape[0])]
    opt.add(delta >= 0)
    for i, var in enumerate(feature_names):
      if var not in anchor_variables: # and importance_dic[var] != 0:
        z3_var = Real(var)
        opt.add((instance[i]) - delta <= z3_var, z3_var <= (instance[i]) + delta)
        # print(f'{instance[i]} - {delta} <= {var}, {var} <= {instance[i]} + {delta}')

    # not D
    self.D = decision_function_expression(self.model, [instance])

    # model
    opt.add(self.T)
    opt.add(Not(self.D))

    # minimize delta
    opt.minimize(delta)
    if opt.check() == sat:
      if verbose:
        for var in opt.model():
          print(var, '=', opt.model()[var])

      print('delta =', opt.model().eval(delta))
    else:
      print("problema inviavel / explicação correta")

# tests

In [16]:
gb_iris = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state = 101)

iris = load_iris()
X_iris, y_iris = iris.data, iris.target

# deixa binario
filter_indices = np.where(np.isin(y_iris, [0, 1]))[0]
X_iris = X_iris[filter_indices]
y_iris = y_iris[filter_indices]

X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=101)

gb_iris.fit(X_iris_train, y_iris_train)

print('Train', sklearn.metrics.accuracy_score(y_iris_train, gb_iris.predict(X_iris_train)))
print('Test', sklearn.metrics.accuracy_score(y_iris_test, gb_iris.predict(X_iris_test)))

gb_iris.predict(X_iris)

Train 1.0
Test 1.0


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [17]:
range(len(iris.feature_names))

range(0, 4)

In [18]:
iris_features_x = [f'x{i}' for i in range(len(iris.feature_names))]
iris_features_x

['x0', 'x1', 'x2', 'x3']

In [19]:
# for i, var in enumerate(iris_features_x):
#   print(var, X_iris[idx][i])

In [20]:
idx = 0
iris_features_x = [f'x{i}' for i in range(len(iris.feature_names))]

anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    gb_iris.classes_,
    iris_features_x,
    X_iris_train,
    categorical_names={})

print('Prediction: ', anchor_explainer.class_names[gb_iris.predict(X_iris[idx].reshape(1, -1))[0]])
exp = anchor_explainer.explain_instance(X_iris[idx], gb_iris.predict, threshold=0.95)

anchor_matrix = []

for name in exp.names():
  tokens = name.split(' ')

  for operator in ['<=', '>=', '==', '<', '>']:
    if operator in name:
      parts = name.split(operator)
      if len(parts) == 2:
        anchor_matrix.append([parts[0].strip(), operator, parts[1].strip()])
        break
anchor_matrix

Prediction:  0


[['x2', '<=', '1.50']]

In [21]:
anchor_expressions = []
for row in anchor_matrix:
  feature = Real(row[0])
  if row[1] == '<=':
    expression = feature <= float(row[2])
  elif row[1] == '>=':
    expression = feature >= float(row[2])
  anchor_expressions.append(expression)
print(anchor_expressions, len(anchor_expressions) == len(anchor_matrix))

[x2 <= 3/2] True


In [22]:
explainer = Explainer(gb_iris, X_iris_train)

In [23]:
instance_explanation = explainer.explain(X_iris_train[idx], 'asc')
instance_explanation

[x2 == 21/5]

In [24]:
print(anchor_expressions)

[x2 <= 3/2]


In [25]:
importances = gb_iris.feature_importances_
importance_dic = {feature: importance for feature, importance in zip(iris_features_x, importances)}
importance_dic

{'x0': np.float64(1.8779019967875804e-16),
 'x1': np.float64(7.844071964451581e-17),
 'x2': np.float64(0.4781893319649887),
 'x3': np.float64(0.5218106680350111)}

In [26]:
x_values_dic = {feature: value for feature, value in zip(iris_features_x, X_iris_train[idx])}
x_values_dic

{'x0': np.float64(5.9),
 'x1': np.float64(3.0),
 'x2': np.float64(4.2),
 'x3': np.float64(1.5)}

In [27]:
opt = Optimize()
delta = Real('delta')

# Adiciona a restrição delta > 0
opt.add(delta >= 0)

anchor_variables = []
for formula in anchor_expressions:
  anchor_variables.append(str(formula.arg(0)))

In [28]:
instance = X_iris_train[idx]

for var in iris_features_x:
  if var not in anchor_variables and importance_dic[var] != 0:
    print(f'{x_values_dic[var]} - {delta} <= {var}, {var} <= {x_values_dic[var]} + {delta}')

5.9 - delta <= x0, x0 <= 5.9 + delta
3.0 - delta <= x1, x1 <= 3.0 + delta
1.5 - delta <= x3, x3 <= 1.5 + delta


In [29]:
instance = X_iris_train[idx]
formula = [Real(f'x{i}') == value for i, value in enumerate(instance)]
print(formula)

[x0 == 59/10, x1 == 3, x2 == 21/5, x3 == 3/2]


In [32]:
print(explainer.T_model)

And(And(Implies(And(x3 <= 3/4),
                o_0_0 == -5405405405405409/2500000000000000),
        Implies(And(x3 > 3/4,
                    x0 <= 2540000057220459/400000000000000),
                o_0_0 == 186046511627907/100000000000000),
        Implies(And(x3 > 3/4,
                    x0 > 2540000057220459/400000000000000,
                    x2 <= 9/2),
                o_0_0 == 186046511627907/100000000000000),
        Implies(And(x3 > 3/4,
                    x0 > 2540000057220459/400000000000000,
                    x2 > 9/2),
                o_0_0 == 18604651162790697/10000000000000000)),
    And(Implies(And(x2 <= 2449999988079071/1000000000000000),
                o_1_0 ==
                -19361926386408657/10000000000000000),
        Implies(And(x2 > 2449999988079071/1000000000000000),
                o_1_0 == 4285970593292383/2500000000000000)),
    And(Implies(And(x3 <= 3/4),
                o_2_0 ==
                -17713961092047843/10000000000000000),
        Implies

In [33]:
expcomp = ExplainerCompleter(gb_iris, anchor_explainer, X_iris, 0)

In [34]:
X_iris_train[3]

array([4.9, 2.4, 3.3, 1. ])

In [35]:
gb_iris.predict([[4.9, 2.4, 3.3, 0.]])

array([1])

In [36]:
expcomp.explain_instance(X_iris_train[3], verbose=True)

o_54_0 = -1252922982135947/1250000000000000
o_3_0 = 1512757987505581/1000000000000000
o_8_0 = -1307641982365609/1000000000000000
o_7_0 = 12909203961986737/10000000000000000
o_27_0 = -10360244289932437/10000000000000000
o_99_0 = -5000129570723073/5000000000000000
o_2_0 = 16018376596421207/10000000000000000
o_29_0 = -10292929521146237/10000000000000000
o_70_0 = -1250588984747267/1250000000000000
o_73_0 = 2000605388901477/2000000000000000
o_89_0 = 5000305487229937/5000000000000000
o_40_0 = 5041377987204111/5000000000000000
o_58_0 = -5007830989488763/5000000000000000
o_62_0 = 10009099244687583/10000000000000000
o_9_0 = -2539865316345887/2000000000000000
o_30_0 = -64151739421477/62500000000000
o_53_0 = 1002241181198809/1000000000000000
o_24_0 = -10492820571035313/10000000000000000
o_75_0 = 5001239054835981/5000000000000000
o_82_0 = 2500307607886103/2500000000000000
delta = 0
o_76_0 = 500112111539463/500000000000000
o_36_0 = -5071621127513253/5000000000000000
o_12_0 = -2965303410892421/25000

In [37]:
expcomp.anchor_expressions

[x2 > 3/2, x1 <= 14/5]

In [38]:
print(expcomp.D)

And((o_0_0 +
     o_1_0 +
     o_2_0 +
     o_3_0 +
     o_4_0 +
     o_5_0 +
     o_6_0 +
     o_7_0 +
     o_8_0 +
     o_9_0 +
     o_10_0 +
     o_11_0 +
     o_12_0 +
     o_13_0 +
     o_14_0 +
     o_15_0 +
     o_16_0 +
     o_17_0 +
     o_18_0 +
     o_19_0 +
     o_20_0 +
     o_21_0 +
     o_22_0 +
     o_23_0 +
     o_24_0 +
     o_25_0 +
     o_26_0 +
     o_27_0 +
     o_28_0 +
     o_29_0 +
     o_30_0 +
     o_31_0 +
     o_32_0 +
     o_33_0 +
     o_34_0 +
     o_35_0 +
     o_36_0 +
     o_37_0 +
     o_38_0 +
     o_39_0 +
     o_40_0 +
     o_41_0 +
     o_42_0 +
     o_43_0 +
     o_44_0 +
     o_45_0 +
     o_46_0 +
     o_47_0 +
     o_48_0 +
     o_49_0 +
     o_50_0 +
     o_51_0 +
     o_52_0 +
     o_53_0 +
     o_54_0 +
     o_55_0 +
     o_56_0 +
     o_57_0 +
     o_58_0 +
     o_59_0 +
     o_60_0 +
     o_61_0 +
     o_62_0 +
     o_63_0 +
     o_64_0 +
     o_65_0 +
     o_66_0 +
     o_67_0 +
     o_68_0 +
     o_69_0 +
     o_70_0 +
     o_71_0 +
  

In [39]:
print(expcomp.T)

And(And(And(Implies(And(x3 <= 3/4),
                    o_0_0 ==
                    -5405405405405409/2500000000000000),
            Implies(And(x3 > 3/4,
                        x0 <=
                        2540000057220459/400000000000000),
                    o_0_0 == 186046511627907/100000000000000),
            Implies(And(x3 > 3/4,
                        x0 >
                        2540000057220459/400000000000000,
                        x2 <= 9/2),
                    o_0_0 == 186046511627907/100000000000000),
            Implies(And(x3 > 3/4,
                        x0 >
                        2540000057220459/400000000000000,
                        x2 > 9/2),
                    o_0_0 ==
                    18604651162790697/10000000000000000)),
        And(Implies(And(x2 <=
                        2449999988079071/1000000000000000),
                    o_1_0 ==
                    -19361926386408657/10000000000000000),
            Implies(And(x2 >
                     

In [40]:
expcomp = ExplainerCompleter(gb_iris, anchor_explainer, X_iris, 0)

for i in range(len(X_iris_train)):
  expcomp.explain_instance(X_iris_train[i])

problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação co

# test breast cancer

In [41]:
gb_cancer = GradientBoostingClassifier(n_estimators=10, max_depth=1, random_state = 101)

cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target

X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=101)

gb_cancer.fit(X_cancer_train, y_cancer_train)

print('Train', sklearn.metrics.accuracy_score(y_cancer_train, gb_cancer.predict(X_cancer_train)))
print('Test', sklearn.metrics.accuracy_score(y_cancer_test, gb_cancer.predict(X_cancer_test)))

gb_cancer.predict(X_cancer)

Train 0.9406593406593406
Test 0.9385964912280702


array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [42]:
idx = 0
cancer_features_x = [f'x{i}' for i in range(len(cancer.feature_names))]

anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    gb_cancer.classes_,
    cancer_features_x,
    X_cancer_train,
    categorical_names={})

In [128]:
set_option(rational_to_decimal=True)

In [146]:
set_option(precision=30)

In [147]:
expcomp = ExplainerCompleter(gb_cancer, anchor_explainer, X_cancer, 0)
expcomp.explain_instance(xctest, test=322)

delta = 322


In [113]:
xctest = X_cancer_train[1] +1
xctest

array([ 13.19    ,  14.29    ,  80.08    , 456.8     ,   1.1066  ,
         1.09509 ,   1.02855 ,   1.02882 ,   1.188   ,   1.06471 ,
         1.2005  ,   1.8163  ,   2.973   ,  16.24    ,   1.006773,
         1.02456 ,   1.01018 ,   1.008094,   1.02662 ,   1.004143,
        14.34    ,  18.81    ,  92.38    , 546.2     ,   1.1427  ,
         1.2585  ,   1.09915 ,   1.08187 ,   1.3469  ,   1.09241 ])

In [125]:
gb_cancer.predict([[ 13.19    ,  14.29    ,  80.08    , 456.8     ,   1.1066  ,
         1.09509 ,   1.02855 ,   1.02882 ,   1.188   ,   1.06471 ,
         1.2005  ,   1.8163  ,   2.973   ,  16.24    ,   1.006773,
         1.02456 ,   1.01018 ,   1.008094,   1.02662 ,   1.004143,
        14.34    ,  18.81    ,  92.38    , 546.2     ,   1.1427  ,
         1.2585  ,   1.09915 ,   1.08187 ,   1.3469  ,   1.09241 ]])

array([1])

In [117]:
print(expcomp.anchor_expressions)

[x22 <= 97.67, x13 <= 17.62]


In [44]:
# expcomp = ExplainerCompleter(gb_cancer, anchor_explainer, X_cancer, 0)

# for i in range(len(X_cancer_train)):
#   expcomp.explain_instance(X_cancer_train[i])

In [149]:
from z3 import *

s = Solver()
x = Real('x')

#
# Here I would establish constraints that limit the
# nunber of decimal places x can have.
# I.E. x -> 1.2; where x can have up to 1 decimal place
#

s.add(x > 1.89, x < 1.91)

if s.check() == unsat:
    print("No solution")

print("x = %s" % str(s.model()[x]))

x = 1.9
